In [1]:
using ConstLab
using Plots
using Voigt
plotly();

In [2]:
include("mises_mixed_hardening.jl")

ATS (generic function with 1 method)

### Parameters

In [3]:
σy =  200.0*10^6
mp = MisesMixedHardMP(
    E = 200.0 * 10^9,
    ν =  0.3,
    σy = σy ,
    H = 0.1 * 200*10^9,
    κ∞ = 0.1 * σy,
    α∞ = 0.1 * σy,
    r = 0.5
);

### Material Status

In [4]:
ms = MisesMixedHardMS()

MisesMixedHardMS
  ₙεₚ: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  ₙσdev: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  ₙαdev: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  ₙκ: 0.0
  ₙμ: 0.0
  loading: false

### Load case: Plain Strain

In [5]:
tend = 1.0
nts = 20
ts = collect(linspace(0, tend, nts))
ε_max = 0.01
εs_pstrain, σs_pstrain, ε_control_pstrain = loadcase(:uniaxial_strain, ε_max, ts, :sym);

In [ ]:
include("mises_mixed_hardening.jl")
@time εs_res_pstrain, σs_res_pstrain, matstats = driver(stress, ATS, ms, mp, ts, εs_pstrain, 
σs_pstrain, ε_control_pstrain);

### Load case: Plain Uniaxial stress

In [7]:
include("mises_mixed_hardening.jl")
nts = 10
ts = collect(linspace(0, 1, nts))
ε_max = 0.01
εs_pstress, σs_pstress, ε_control_pstress = loadcase(:uniaxial_stress, ε_max, ts, :unsym);
εs_res_pstress, σs_res_pstress, matstats = driver(stress, ATS, ms, mp, ts, εs_pstress, 
                                                  σs_pstress, ε_control_pstress, ftol = 1e-5);

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     1.282051e+08              NaN
     1     1.282051e+08     1.165517e-23
     2     1.282051e+08     4.662069e-23
     3     1.282051e+08     1.864828e-22
     4     1.282051e+08     7.459310e-22
     5     1.282051e+08     2.983724e-21
     6     1.282051e+08     1.193490e-20
     7     1.282050e+08     4.773959e-20
     8     1.282049e+08     1.909583e-19
     9     1.282047e+08     7.638334e-19
    10     1.282042e+08     3.055334e-18
    11     1.282032e+08     1.222133e-17
    12     1.282013e+08     4.888534e-17
    13     1.281975e+08     1.955413e-16
    14     1.281899e+08     7.821654e-16
    15     1.281747e+08     3.128662e-15
    16     1.281443e+08     1.251465e-14
    17     1.280834e+08     5.005858e-14
    18     1.279617e+08     2.002343e-13
    19     1.277183e+08     8.009373e-13
    20     1.272316e+08     3.203749e-12
    21     1.262580e+08     1.281500e-11
    22     1.24310

### Plotting

In [ ]:
vm(x) = sqrt(3/2) * vnorm(dev(x));

In [ ]:
#p = plot(εs_res_pstrain, σs_res_pstrain, matstats; x_mod = 1, y_mod = 1);
p = plot()
p = plot!(p, vec(εs_res_pstress[1,:]), 
          Float64[vm(σs_res_pstrain[:,i]) for i = 1:nts],
          label="Plane strain");
p = plot!(p, vec(εs_res_pstress[1,:]), 
             Float64[vm(σs_res_pstress[:,i]) for i = 1:nts],
             label="Plane stress");
p = plot!(p, vec(εs_res_pstress[1,:]), 
             (mp.κ∞ + mp.α∞ + mp.σy) * ones(ts),
             label="Saturation stress")
xlabel!(p, "ε_11"); ylabel!(p, "σ"); title!(p, "Uniaxial strain vs stress")